In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import bs4 as BeautifulSoup

/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [14]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the player statistics page
driver.get("https://www.nba.com/stats/players/traditional")

# Explicit wait to ensure the table is loaded
wait = WebDriverWait(driver, 10)

general_players_table_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/'

# Esperar explícitamente a que el enlace esté presente
try:
    # Esperar a que aparezca el botón de aceptar cookies
    cookie_accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
    )
    cookie_accept_button.click()  # Hacer clic en "Acepto"
    print("Cookies banner closed.")
except Exception as e:
    print("No cookies banner found or other issue:", e)

try:
    # Esperar a que el enlace sea clicable
    link_xpath = general_players_table_xpath + 'tbody/tr[1]/td[11]/a'
    link_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, link_xpath))
    )

    link_url = link_element.get_attribute("href")
    print(f"Link found: {link_element.get_attribute('href')}")
    driver.get(link_url)

except Exception as e:
    print(f"Error: {e}")

time.sleep(5)
driver.quit()

Cookies banner closed.
Link found: https://www.nba.com/stats/events/?CFID=&CFPARAMS=&ContextMeasure=FGA&GameID=&PlayerID=203999&Season=2024-25&SeasonType=Regular%20Season&TeamID=1610612743&flag=3&sct=plot&section=game
